## Revenue Generation in Uniswap

In [1]:
import os
import copy
import numpy as np
import pandas as pd
import time
import datetime
import math
import matplotlib.pyplot as plt

In [2]:
from uniswappy import *

In [3]:
user_nm = 'user0'
dai_amount = 10000
sys_amount = 10000

fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

In [4]:
dai1 = ERC20("DAI", "0x111")
sys1 = ERC20("SYS", "0x09")
exchg_data = UniswapExchangeData(tkn0 = sys1, tkn1 = dai1, symbol="LP",
                                   address="0x011", version = 'V3',
                                   tick_spacing = tick_spacing,
                                   fee = fee)

factory = UniswapFactory("SYS pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, dai_amount, sys_amount, lwr_tick, upr_tick)
lp.summary()

Exchange SYS-DAI (LP)
Real Reserves:   SYS = 10000.0, DAI = 10000.0
Gross Liquidity: 10000.0 



In [5]:
LPQuote(False).get_amount_from_lp(lp, sys1, 1, lwr_tick, upr_tick)

1.9969005990711928

### Liquidity leak test-batch #1

In [6]:
lp.last_liquidity_deposit

10000.0

In [7]:
lp.summary()
dai_amt = LPQuote(False).get_amount_from_lp(lp, dai1, 1, lwr_tick, upr_tick)
print('Pre-tests:')
print('- 1 LP token is {:.6f} DAI'.format(dai_amt))
print('- we are starting with {:.6f} LP token'.format(lp.get_liquidity()))

Exchange SYS-DAI (LP)
Real Reserves:   SYS = 10000.0, DAI = 10000.0
Gross Liquidity: 10000.0 

Pre-tests:
- 1 LP token is 1.996901 DAI
- we are starting with 10000.000000 LP token


#### Test 1: SwapDeposit

In [8]:
requested_liquidity_in = 500
sys_before_test = lp.get_reserve(sys1)
sys_settlement_amt = LPQuote(False).get_amount_from_lp(lp, sys1, requested_liquidity_in, lwr_tick, upr_tick)
dep = SwapDeposit().apply(lp, sys1, user_nm, sys_settlement_amt, lwr_tick, upr_tick)
get_deposit = LPQuote(False).get_amount_from_lp(lp, sys1, lp.get_last_liquidity_deposit(), lwr_tick, upr_tick)
lp.summary()

Exchange SYS-DAI (LP)
Real Reserves:   SYS = 10973.646046926347, DAI = 10000.0
Gross Liquidity: 10474.83520392498 



In [9]:
out = WithdrawSwap().apply(lp, sys1, user_nm, get_deposit, lwr_tick, upr_tick)
lp.summary()

Exchange SYS-DAI (LP)
Real Reserves:   SYS = 10002.78890824099, DAI = 10000.0
Gross Liquidity: 10000.0 



In [10]:
lp.summary()
dai_amt = LPQuote(False).get_amount_from_lp(lp, dai1, 1, lwr_tick, upr_tick)
sys_after_test = lp.get_reserve(sys1)
print('Post test #1:')
print('- 1 LP token is {:.6f} DAI'.format(dai_amt))
print('- we have {:.6f} total LP token'.format(lp.get_liquidity()))
print('- we made a profit of {:.6f} SYS post swapping'.format(sys_after_test-sys_before_test))

Exchange SYS-DAI (LP)
Real Reserves:   SYS = 10002.78890824099, DAI = 10000.0
Gross Liquidity: 10000.0 

Post test #1:
- 1 LP token is 1.996629 DAI
- we have 10000.000000 total LP token
- we made a profit of 2.788908 SYS post swapping


#### Test 2: Multiple WithdrawSwap / SwapDeposit

In [11]:
# TKN_amt = TokenDeltaModel(1000)
# N = 100
# for k in range(N):
#     Swap().apply(lp, sys1, user_nm, TKN_amt.delta())
#     Swap().apply(lp, dai1, user_nm, TKN_amt.delta())

# LPQuote(False).get_amount_from_lp(lp, sys1, 1, lwr_tick, upr_tick)

In [12]:
sys_before_test = lp.get_reserve(sys1)
N = 100
for k in range(N):
    requested_liquidity_out = 500
    sys_settlement_amt = LPQuote(False).get_amount_from_lp(lp, sys1, requested_liquidity_in, lwr_tick, upr_tick)
    dep = SwapDeposit().apply(lp, sys1, user_nm, sys_settlement_amt, lwr_tick, upr_tick)
    get_deposit = LPQuote(False).get_amount_from_lp(lp, sys1, lp.get_last_liquidity_deposit(), lwr_tick, upr_tick)
    out = WithdrawSwap().apply(lp, sys1, user_nm, get_deposit, lwr_tick, upr_tick)

In [13]:
lp.summary()
dai_amt = LPQuote(False).get_amount_from_lp(lp, dai1, 1, lwr_tick, upr_tick)
sys_after_test = lp.get_reserve(sys1)
print('Post test #2:')
print('- 1 LP token is {:.6f} DAI'.format(dai_amt))
print('- we have {:.6f} LP token remaining'.format(lp.get_liquidity()))
print('- we made a profit of {:.6f} SYS post swapping'.format(sys_after_test-sys_before_test))

Exchange SYS-DAI (LP)
Real Reserves:   SYS = 10283.603944861577, DAI = 10000.0
Gross Liquidity: 10000.000000000007 

Post test #2:
- 1 LP token is 1.969658 DAI
- we have 10000.000000 LP token remaining
- we made a profit of 280.815037 SYS post swapping


In [14]:
LPQuote(False).get_amount_from_lp(lp, sys1, 1, lwr_tick, upr_tick)

2.0245197624797155